# Neural Network to Distinguish Power Density Graphs

This repository contains the code to train and export a neural network that takes in an input of a Power Density Function (PDF) file and output a value between 0 and 1. A value closer to 1 indicates that the function contains inaccuracies that may need to be reviewed, and values closer to 0 means that there are no apparent inaccuracies.

### Prerequisites

The following code heavily utilizes the Pytorch library. If unfamiliar with Pytorch or fundamental machine learning concepts, please review the following links before attempting to modify the code:

[3Blue1Brown's Introduction to Deep Learning](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)

[Pytorch Tutorial - What is torch.nn really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html)

[Pytorch Tutorial - Beginner Basics](https://pytorch.org/tutorials/beginner/basics/intro.html)

In [2]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import os

from tqdm import tqdm

import sys

### Loading Data

In [3]:
# For debugging purposes - prints the full values of numpy arrays and pytorch tensors

# np.set_printoptions(threshold=sys.maxsize)
# torch.set_printoptions(threshold=sys.maxsize)

The following code creates a [custom dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) that is used to load data for both the training and validation set. The current repository must contain a folder for training data and a folder for validation, with each file named after numbers starting from 0 (0, 1, 2...).

Each PDF file inside both the training and validation data must follow the format of 

For the sake of simplicity in labeling, 

In [ ]:
class PDFDataset(Dataset):
    def __init__(self, is_training):
        file_path = "training" if is_training else "validation"
        self.length = len(os.listdir(file_path))
        
        self.data = {}

        for i in tqdm(range(self.length)):
            arr = np.fromfile(f"{file_path}/{i}", sep=" ")[2::3]
            self.data[i] = torch.from_numpy(arr).float()
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i], torch.tensor(i % 2).reshape((1, )).float()

In [ ]:
training_data = PDFDataset(False)
validation_data = PDFDataset(True)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)
test_dataloader = DataLoader(validation_data, batch_size=2)

### Neural Network

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(122 * 151, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)

In [ ]:
model = NeuralNetwork()

### Training

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backward propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
learning_rate = 1e-2
batch_size = 2
epochs = 10

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t + 1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")